In [1]:
import pandas as pd

In [2]:
info = pd.read_csv("/home/SENT_mails_info.csv")
info

,date,subject,sender,receiver,message_id,label,next_page_id
0,"Sat, 12 Nov 2022 23:01:21 +0530",Re: Work-Life Balance at Decision Point,Tushar Sharma <tushar5353@gmail.com>,akshita@decisionpointanalytics.in,1846ce5671e870a1,SENT,8838806045064602880
1,"Fri, 11 Nov 2022 13:00:07 +0530",Staff Data Engineer,Tushar Sharma <tushar5353@gmail.com>,ritishree.geda@innovaccer.com,184659890df3b4cd,SENT,8838806045064602880
2,"Tue, 8 Nov 2022 17:46:24 +0530",updated resume,Tushar Sharma <tushar5353@gmail.com>,chary.vootla@sawfe.com,184572b9b9851bab,SENT,8838806045064602880
3,"Tue, 8 Nov 2022 10:07:27 +0530",Data engineer,Tushar Sharma <tushar5353@gmail.com>,hr@sawfe.com,184558767c0f0dab,SENT,8838806045064602880
4,"Tue, 8 Nov 2022 08:23:30 +0530",Data Engineer,Tushar Sharma <tushar5353@gmail.com>,rajkumar@firstconnectsolutions.com,18455283c0cfec30,SENT,8838806045064602880
...,...,...,...,...,...,...,...
868,"Sat, 17 Mar 2012 21:54:22 +0530",resume,tushar sharma <tushar5353@gmail.com>,opstushar@yahoo.com,1362179564885e77,SENT,4836019439571374504
869,"Sun, 11 Sep 2011 10:04:18 +0530",resume,tushar sharma <tushar5353@gmail.com>,poonam@anaadihsoft.com,13256c3b26e27ea2,SENT,4836019439571374504
870,"Fri, 15 Jul 2011 22:37:01 +0530",reply,tushar sharma <tushar5353@gmail.com>,tutorresource@gmail.com,1312ec3fcc2014b3,SENT,4836019439571374504
871,"Fri, 15 Jul 2011 22:19:48 +0530",reply,tushar sharma <tushar5353@gmail.com>,tutorresource@gmail.com,1312eb43a01748c1,SENT,4836019439571374504


In [5]:
info['date'] = pd.to_datetime(info['date'])

In [14]:
info['sender'] = info['sender'].apply(extract_emails)
info

,date,subject,sender,receiver,message_id,label,next_page_id
0,2022-11-12 23:01:21+05:30,Re: Work-Life Balance at Decision Point,tushar5353@gmail.com,akshita@decisionpointanalytics.in,1846ce5671e870a1,SENT,8838806045064602880
1,2022-11-11 13:00:07+05:30,Staff Data Engineer,tushar5353@gmail.com,ritishree.geda@innovaccer.com,184659890df3b4cd,SENT,8838806045064602880
2,2022-11-08 17:46:24+05:30,updated resume,tushar5353@gmail.com,chary.vootla@sawfe.com,184572b9b9851bab,SENT,8838806045064602880
3,2022-11-08 10:07:27+05:30,Data engineer,tushar5353@gmail.com,hr@sawfe.com,184558767c0f0dab,SENT,8838806045064602880
4,2022-11-08 08:23:30+05:30,Data Engineer,tushar5353@gmail.com,rajkumar@firstconnectsolutions.com,18455283c0cfec30,SENT,8838806045064602880
...,...,...,...,...,...,...,...
868,2012-03-17 21:54:22+05:30,resume,tushar5353@gmail.com,opstushar@yahoo.com,1362179564885e77,SENT,4836019439571374504
869,2011-09-11 10:04:18+05:30,resume,tushar5353@gmail.com,poonam@anaadihsoft.com,13256c3b26e27ea2,SENT,4836019439571374504
870,2011-07-15 22:37:01+05:30,reply,tushar5353@gmail.com,tutorresource@gmail.com,1312ec3fcc2014b3,SENT,4836019439571374504
871,2011-07-15 22:19:48+05:30,reply,tushar5353@gmail.com,tutorresource@gmail.com,1312eb43a01748c1,SENT,4836019439571374504


In [13]:
def extract_emails(row):
    regex = r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b'
    emails = re.findall(r'\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b', row)
    emails = ",".join(emails)
    return emails

In [10]:
extract_emails("safasdf@sdfds.com, asdfsadf@d.com, sdfasdf")

'safasdf@sdfds.com,asdfsadf@d.com'